# Sample Code

## 載入資料

In [1]:
import pandas as pd
import numpy as np

/Users/wei/.virtualenvs/py3/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
## Sample Data

metadata = pd.DataFrame({
    'asin': ['A1', 'A2', 'AB3', 'B4', 'B5'],
    'title': ['Hello World', 'Hello Python', 'Python Tutorial', 'Java Tutorial', 'C++ Tutorial']
})
metadata

,asin,title
0,A1,Hello World
1,A2,Hello Python
2,AB3,Python Tutorial
3,B4,Java Tutorial
4,B5,C++ Tutorial


In [3]:
ratings = pd.DataFrame([
    {'asin': 'A1','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'A2','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'AB3','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'B4','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'AB3','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    {'asin': 'A1','reviewerID': 'USER3', 'overall': 5, 'unixReviewTime': 1424304000 ,'DATE': '2015-02-19'},
    
    {'asin': 'A1','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'A2','reviewerID': 'USER1', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'B5','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'AB3','reviewerID': 'USER2', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},
    {'asin': 'A2','reviewerID': 'USER3', 'overall': 5, 'unixReviewTime': 1537315200 ,'DATE': '2018-09-19'},

])
ratings

,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,A2,USER1,5,1424304000,2015-02-19
2,AB3,USER1,5,1424304000,2015-02-19
3,B4,USER2,5,1424304000,2015-02-19
4,AB3,USER2,5,1424304000,2015-02-19
5,A1,USER3,5,1424304000,2015-02-19
6,A1,USER1,5,1537315200,2018-09-19
7,A2,USER1,5,1537315200,2018-09-19
8,B5,USER2,5,1537315200,2018-09-19
9,AB3,USER2,5,1537315200,2018-09-19


In [4]:
metadata.head()

,asin,title
0,A1,Hello World
1,A2,Hello Python
2,AB3,Python Tutorial
3,B4,Java Tutorial
4,B5,C++ Tutorial


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,A2,USER1,5,1424304000,2015-02-19
2,AB3,USER1,5,1424304000,2015-02-19
3,B4,USER2,5,1424304000,2015-02-19
4,AB3,USER2,5,1424304000,2015-02-19


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin', 'overall']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: dict(zip(rating['asin'], rating['overall'])) for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [8]:
ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,A1,USER1,5,1424304000,2015-02-19
1,A2,USER1,5,1424304000,2015-02-19
2,AB3,USER1,5,1424304000,2015-02-19
3,B4,USER2,5,1424304000,2015-02-19
4,AB3,USER2,5,1424304000,2015-02-19
5,A1,USER3,5,1424304000,2015-02-19


In [9]:
ratings_testings

,asin,reviewerID,overall,unixReviewTime,DATE
6,A1,USER1,5,1537315200,2018-09-19
7,A2,USER1,5,1537315200,2018-09-19
8,B5,USER2,5,1537315200,2018-09-19
9,AB3,USER2,5,1537315200,2018-09-19
10,A2,USER3,5,1537315200,2018-09-19


In [10]:
users

['USER1', 'USER2', 'USER3']

In [11]:
ratings_testings_by_user

{'USER1': {'A1': 5, 'A2': 5}, 'USER2': {'B5': 5, 'AB3': 5}, 'USER3': {'A2': 5}}

In [12]:
pd.DataFrame(ratings_testings_by_user)

,USER1,USER2,USER3
A1,5.0,NaN,NaN
A2,5.0,NaN,5.0
B5,NaN,5.0,NaN
AB3,NaN,5.0,NaN


## 產生推薦

In [13]:
import math
from collections import defaultdict

def defItemIndex(DictUser):
    DictItem = defaultdict(defaultdict) 
    ## 遍历每个用户
    for key in DictUser: 
        ##遍 历用户k的购买记录
        for i in DictUser[key]:
            DictItem[i].setdefault(key, 0)
            DictItem[i][key] += DictUser[key][i]
    return DictItem
    
# 以 User 的角度計算 User 間的相似性    
def defUserSimilarity(DictItem):  
    N=dict()   #用户购买的数量
    C=defaultdict(defaultdict)  
    W=defaultdict(defaultdict)  
    ## 遍历每个物品
    for key in DictItem: 
        ## 遍历用户k购买过的书
        for i in DictItem[key]:  
            # i[0]表示用户的id ，如果未计算过，则初始化为0
            if i not in N.keys():  
                N[i]=0  
            N[i]+=1     
            ## (i,j)是物品k同时被购买的用户两两匹配对          
            for j in DictItem[key]:  
                if i==j:  
                    continue    
                if j not in C[i].keys():  
                    C[i][j]=0  
                #C[i[0]][j[0]]表示用户i和j购买同样书的数量  
                C[i][j]+=1      
    for i,related_user in C.items():  
        for j,cij in related_user.items():  
            W[i][j]=cij/math.sqrt(N[i]*N[j])   
    return W   

# 找出特定 User 的未知 Item 的分數
def RecommendUserCF(train, user_id, W):  
    user_rating = W[user_id] 
    user_rating = list(user_rating.items())
    user_rating.sort(key=lambda val:val[1],reverse=True)#按照评分排序
    
    rank = dict()  
    try:
        for i in train[user_rating[0][0]]:
            rank[i] = train[user_rating[0][0]][i]

        return dict(sorted(list(rank.items()), key=lambda val:val[1],reverse=True))
    except:
        return {}

In [14]:
pd.DataFrame(ratings_testings_by_user)
pd.DataFrame(defItemIndex(ratings_testings_by_user))
# = pd.DataFrame(ratings_testings_by_user).T

,A1,A2,B5,AB3
USER1,5.0,5.0,NaN,NaN
USER3,NaN,5.0,NaN,NaN
USER2,NaN,NaN,5.0,5.0


In [15]:
W = defUserSimilarity(defItemIndex(ratings_testings_by_user))
pd.DataFrame(W)

,USER1,USER3
USER3,0.707107,NaN
USER1,NaN,0.707107


In [16]:
RecommendUserCF(ratings_testings_by_user, 'USER1', W)

{'A2': 5}

In [17]:
RecommendUserCF(ratings_testings_by_user, 'USER2', W)

{}

In [18]:
RecommendUserCF(ratings_testings_by_user, 'USER3', W)

{'A1': 5, 'A2': 5}

In [19]:
def recommender(training_data, users=[], k=2):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    cf-item-based
    '''
    ratings_trainings = training_data
    recommendations = {user: list(RecommendUserCF(ratings_testings_by_user, user, W).keys()) for user in users}

    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'USER1': ['A2'], 'USER2': [], 'USER3': ['A1', 'A2']}

## 結果評估

In [20]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.4

In [21]:
ratings_by_user

{'USER1': ['A2'], 'USER2': [], 'USER3': ['A1', 'A2']}

In [22]:
ratings_testings_by_user

{'USER1': {'A1': 5, 'A2': 5}, 'USER2': {'B5': 5, 'AB3': 5}, 'USER3': {'A2': 5}}